In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import glob


In [2]:
path = 'nlm_genes_tissue_label'
files = glob.glob(path + "/*.csv")


In [10]:
files[0]

'nlm_genes_tissue_label/nlm_genes_tissue_label_set_1.csv'

In [44]:
# Load random set of BSM-ID's
bsm_genes = pd.read_csv('bsm_2k_random.csv')
bsm_genes.drop(['cid'], inplace=True, axis=1)
bsm_genes.head()
og_bsm_columns = bsm_genes.columns
bsm_for_lables = bsm_genes.copy()

In [45]:
for file in files:
    print('file is',file)
    nlm_genes = pd.read_csv(file)
    top_10_tissues_nlm = nlm_genes[nlm_genes['tissue_name'].map(nlm_genes['tissue_name'].value_counts()) > 510]
    #print(top_10_tissues_nlm['tissue_name'].value_counts())
    og_labls = top_10_tissues_nlm['tissue_name'].value_counts().index.to_list()
    #print(top_10_tissues_nlm.head())
    #break
    
    #print('null_bvalues', top_10_tissues_nlm.isnull().sum())
    #break
    # rename the gsm and bsm columns
    rename_cols = {}
    for old_name, new_name in zip(bsm_genes.columns, top_10_tissues_nlm.drop(['tissue_name', 'cid'], axis=1).columns):
        #print(old_name, new_name)
        rename_cols[old_name] = new_name
    bsm_genes = bsm_genes.rename(columns=rename_cols)
    combined_data = pd.concat([top_10_tissues_nlm.drop(['tissue_name', 'cid'], axis=1), bsm_genes], ignore_index=True)
    
    print('NULL',combined_data.isnull().sum())
    kdf = combined_data 
    
    # Performing Kmeans
    from sklearn.cluster import KMeans

    # Number of clusters
    kmeans = KMeans(n_clusters=10, random_state=12)
    # Fitting the input data
    kmeans = kmeans.fit(kdf)
    # Getting the cluster labels
    labels = kmeans.predict(kdf)
    # Centroid values
    centroids = kmeans.cluster_centers_
    
        # Datapoints count in each Clusters
    uniqueValues, occurCount = np.unique(labels, return_counts=True)

    print("Unique Values : " , uniqueValues)
    print("Occurrence Count : ", occurCount)

    sort_labels = np.argsort(-occurCount)[:10]

    ## Label mapping to cluster
    clus_lab = {}

    ## sort_lables and og_labls contain the clusters and labels in decresing order
    for clus, lab in zip(sort_labels, og_labls):
        clus_lab[clus] = lab
    print(clus_lab)
    rev_clus_lab = {v: k for k, v in clus_lab.items()}
    print(rev_clus_lab)
    
    
    # Encode labels
    encoded_lab = []
    for i in top_10_tissues_nlm['tissue_name']:
        encoded_lab.append(rev_clus_lab[i])
        
    set_no = file.split('/')[-1].split('nlm_genes_tissue_label_')[-1].split('.csv')[0]
    print('=======================\n')
    print(f'set is {set_no}')
    print(f'{set_no}, {accuracy_score(encoded_lab[:len(top_10_tissues_nlm)], labels[:len(top_10_tissues_nlm)])*100},%')
    
    ## labels for the bsm genes predict
    bsm_labels = []
    for lab in labels[len(top_10_tissues_nlm):]:
        #print(clus_lab[lab])
        bsm_labels.append(clus_lab[lab])

    bsm_for_lables[set_no] = bsm_labels
    print('**************************')

    
    
    
    
    
    
    
    #kdf = top_10_tissues_nlm.drop(['tissue_name'])


    
    

file is nlm_genes_tissue_label/nlm_genes_tissue_label_set_1.csv
NULL 202207_at                  0
203126_at                  0
203573_s_at                0
203594_at                  0
204017_at                  0
                          ..
59999_at                   0
63825_at                   0
AFFX-HSAC07/X00351_5_at    0
AFFX-r2-Bs-lys-3_at        0
AFFX-r2-Ec-bioD-3_at       0
Length: 978, dtype: int64
Unique Values :  [0 1 2 3 4 5 6 7 8 9]
Occurrence Count :  [ 287  727 1011 2737 1024  629  816  641  590 1531]
{3: 'Leukemia patient sample', 9: 'HumanAorticEndothelialCells', 4: 'pre-treatment bone marrow', 2: 'Breast', 6: 'Male/Female; Normal/Diseased', 1: 'Leukemic cells obtained from bone marrow or blood at diagnosis', 7: 'PBMC', 5: 'Breast Tumor', 8: 'Blasts and mononuclear cells, AML patient', 0: 'primary colorectal adenocarcinoma'}
{'Leukemia patient sample': 3, 'HumanAorticEndothelialCells': 9, 'pre-treatment bone marrow': 4, 'Breast': 2, 'Male/Female; Normal/Diseased': 6

NULL 200618_at                      0
201924_at                      0
202401_s_at                    0
202441_at                      0
204952_at                      0
                              ..
60794_f_at                     0
AFFX-HUMISGF3A/M97935_MA_at    0
AFFX-LysX-M_at                 0
AFFX-r2-Bs-dap-5_at            0
AFFX-TrpnX-5_at                0
Length: 978, dtype: int64
Unique Values :  [0 1 2 3 4 5 6 7 8 9]
Occurrence Count :  [1016 1245 1527  581  750 1047 2130  288  596  813]
{6: 'Leukemia patient sample', 2: 'HumanAorticEndothelialCells', 1: 'pre-treatment bone marrow', 5: 'Breast', 0: 'Male/Female; Normal/Diseased', 9: 'Leukemic cells obtained from bone marrow or blood at diagnosis', 4: 'PBMC', 8: 'Breast Tumor', 3: 'Blasts and mononuclear cells, AML patient', 7: 'primary colorectal adenocarcinoma'}
{'Leukemia patient sample': 6, 'HumanAorticEndothelialCells': 2, 'pre-treatment bone marrow': 1, 'Breast': 5, 'Male/Female; Normal/Diseased': 0, 'Leukemic cells ob

NULL 201207_at              0
201794_s_at            0
202369_s_at            0
203905_at              0
204752_x_at            0
                      ..
61297_at               0
79005_at               0
AFFX-PheX-3_at         0
AFFX-r2-Bs-lys-3_at    0
AFFX-TrpnX-5_at        0
Length: 978, dtype: int64
Unique Values :  [0 1 2 3 4 5 6 7 8 9]
Occurrence Count :  [ 751 1045 1016  815 1244 1509  288 2132  574  619]
{7: 'Leukemia patient sample', 5: 'HumanAorticEndothelialCells', 4: 'pre-treatment bone marrow', 1: 'Breast', 2: 'Male/Female; Normal/Diseased', 3: 'Leukemic cells obtained from bone marrow or blood at diagnosis', 0: 'PBMC', 9: 'Breast Tumor', 8: 'Blasts and mononuclear cells, AML patient', 6: 'primary colorectal adenocarcinoma'}
{'Leukemia patient sample': 7, 'HumanAorticEndothelialCells': 5, 'pre-treatment bone marrow': 4, 'Breast': 1, 'Male/Female; Normal/Diseased': 2, 'Leukemic cells obtained from bone marrow or blood at diagnosis': 3, 'PBMC': 0, 'Breast Tumor': 9, 'Blasts

In [47]:
bsm_for_lables.to_csv('bsm_with_labels.csv', index=False)